In [1]:
import numpy as np 
import pandas as pd
import os
import io
import glob
import re
import sys
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import animation, rc
import plotly.express as px
import plotly.graph_objects as go

import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.io as pio

import geopandas as gpd

import numpy as np 
import pandas as pd
import os
import glob
import re
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import animation, rc
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import plotly.express as px
import folium
from folium import plugins
import branca
import json
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.io as pio
import matplotlib.pylab as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

pio.renderers.default = "notebook_connected"

In [62]:
import os
os.getcwd()
os.chdir("C:/Users/JYW/Desktop/Github/Data-Science-for-COVID-19")

files = glob.glob('dataset/*.csv')
data = []
for i in files :
    i = re.sub("dataset\\\\", "", i)
    data.append(re.sub(".csv","",i))
for i in range(len(files)) :
    globals()[data[i]] = pd.read_csv(files[i])
data

['Case',
 'Latitude',
 'Longitude',
 'PatientInfo',
 'PatientRoute',
 'Policy',
 'Region',
 'SearchTrend',
 'SeoulFloating',
 'SFedit',
 'Time',
 'TimeAge',
 'TimeGender',
 'TimeProvince',
 'Weather']

In [54]:
twtinfo = PatientInfo[PatientInfo.age == '20s']
twtinfo.head()
len(twtinfo)
twtinfo.infection_case.value_counts()

overseas inflow                    233
contact with patient               143
etc                                114
Shincheonji Church                  41
Onchun Church                       18
Guro-gu Call Center                 13
Dongan Church                        8
Milal Shelter                        3
Eunpyeong St. Mary's Hospital        2
Seongdong-gu APT                     2
Changnyeong Coin Karaoke             2
River of Grace Community Church      1
gym facility in Cheonan              1
Cheongdo Daenam Hospital             1
gym facility in Sejong               1
Name: infection_case, dtype: int64

해외 유입, 사람 접촉이 제일 많다.

## infection_case를 시간별로 보자

In [69]:
twtinfo['infection_case'] = twtinfo['infection_case'].fillna("NA")
twtinfo.infection_case.isnull().sum()

0

infection_case가 NA인 애들도 있다. NA로 표시


In [70]:
twtinfo = twtinfo.sort_values(by=['confirmed_date'], axis=0)
counts = pd.get_dummies(twtinfo['infection_case']).cumsum()
counts = twtinfo[["confirmed_date","infection_case"]].join(counts)

counts["summary_infect"] = [ list(counts[list(counts.infection_case)[i]])[i] for i in range(0, len(counts)) ]
counts = counts.drop_duplicates(["confirmed_date","infection_case"], keep="last")
counts = counts.sort_values(by=['confirmed_date'], axis=0)

counts.tail(3)

,confirmed_date,infection_case,Changnyeong Coin Karaoke,Cheongdo Daenam Hospital,Dongan Church,Eunpyeong St. Mary's Hospital,Guro-gu Call Center,Milal Shelter,NA,Onchun Church,River of Grace Community Church,Seongdong-gu APT,Shincheonji Church,contact with patient,etc,gym facility in Cheonan,gym facility in Sejong,overseas inflow,summary_infect
936,2020-04-18,overseas inflow,2,1,8,2,13,3,207,18,1,2,41,143,114,1,1,231,231
752,2020-04-18,etc,2,1,8,2,13,3,207,18,1,2,41,143,114,1,1,230,114
937,2020-04-19,overseas inflow,2,1,8,2,13,3,207,18,1,2,41,143,114,1,1,233,233


In [59]:
fig = go.Figure()
fig = px.line(counts, x='confirmed_date', y='summary_infect',
              color="infection_case",
              line_group="infection_case", hover_name="infection_case")
fig.update_layout(title='20s infection_case' , showlegend=True)

fig.show()

해외유입이 급격하게 증가함 \
생각보다 접촉은 완만한 그래프 \
NA로 표시되었던게 엄청나게 많았다.(보라색) (3월초에)\
쟤들은 뭐지..?

## 다른 나이대는?

In [71]:
nottwtinfo = PatientInfo[PatientInfo.age != '20s']
nottwtinfo.infection_case.isnull().sum()

0

In [72]:
nottwtinfo = nottwtinfo.sort_values(by=['confirmed_date'], axis=0)
counts = pd.get_dummies(nottwtinfo['infection_case']).cumsum()
counts = nottwtinfo[["confirmed_date","infection_case"]].join(counts)

counts["summary_infect"] = [ list(counts[list(counts.infection_case)[i]])[i] for i in range(0, len(counts)) ]
counts = counts.drop_duplicates(["confirmed_date","infection_case"], keep="last")
counts = counts.sort_values(by=['confirmed_date'], axis=0)

counts.tail(3)

,confirmed_date,infection_case,Bonghwa Pureun Nursing Home,Changnyeong Coin Karaoke,Cheongdo Daenam Hospital,Dongan Church,Eunpyeong St. Mary's Hospital,Geochang Church,Guro-gu Call Center,Gyeongsan Cham Joeun Community Center,...,Pilgrimage to Israel,Seongdong-gu APT,Shincheonji Church,Suyeong-gu Kindergarten,contact with patient,etc,gym facility in Cheonan,gym facility in Sejong,overseas inflow,summary_infect
3009,2020-04-18,contact with patient,31,2,20,9,14,6,99,10,...,2,11,66,3,805,449,29,3,333,805
1720,2020-04-19,contact with patient,31,2,20,9,14,6,99,10,...,2,11,66,3,806,450,29,3,334,806
2925,NaN,etc,31,2,20,9,14,6,99,10,...,2,11,66,3,806,453,29,3,334,453


In [73]:
fig = go.Figure()
fig = px.line(counts, x='confirmed_date', y='summary_infect',
              color="infection_case",
              line_group="infection_case", hover_name="infection_case")
fig.update_layout(title='All age infection_case' , showlegend=True)

fig.show()

20대를 제외한 연령대에서의 그래프 추이\
주변인 접촉이 압도적으로 많음 -> 20대가 생각없이 다른 사람들 만나고 다닌건 아닌 거 같다 \
여기도 NA가 3월초에 급격히 증가. 쟨 도대체 뭘까\
해외유입은 4등정도. 20대가 해외유입으로 걸리는게 많긴 많구나.